In [ ]:
#r "nuget:TaskBuilder.fs,2.1.0"
#r "TakensTheorem.Core.dll"
open TakensTheorem.Core
open KaggleClient

In [ ]:
//open System
open System.Net.Http
open FSharpKernelHelpers 

In [ ]:
[<RequireQualifiedAccess>]
module ProgressBar =
    let Create className statusText =     
        sprintf "<div class='%s'> 
                    <div class='progress'> 
                        <div class='progress-bar' role='progressbar' 
                              style='width: 1%%;' aria-valuenow='1' aria-valuemin='0' 
                              aria-valuemax='100'>1%%</div>                     
                     </div> 
                     <div><code class='progress-status'>%s</code></div> 
                 </div>" className statusText 
        |> HtmlString 
    
    let CreateWithPocketView (className:string) (statusText: string) =
        div.["class",className].innerHTML [
            div.["class","progress"].innerHTML (
                div.["class", "progress-bar"].["role", "progressbar"]
                   .["style", "width: 1%"].["aria-valuenow", 1]
                   .["aria-valuemin", 0 ].["aria-valuemax", 100] )
            div.innerHTML (code.["class","progress-status"].innerHTML statusText) ] 

    let Update className status value =
        let str = StringBuilder()

        sprintf "$('.%s .progress-status').text('%s');" className status
        |> str.AppendLine

        sprintf "$('.%s .progress-bar')
                    .css('width','%.02f%%')
                    .prop('aria-valuenow',%.02f)
                    .text('%.02f%%');" className value value value
        |> str.AppendLine

        str.ToString() |> Javascript    
  

In [ ]:
let Report() =  
    let guid = Guid.NewGuid()
    let pbarClass = sprintf "bar-%O" guid
    let markerClass = sprintf "code-%O" guid

    ProgressBar.CreateWithPocketView pbarClass ""
    |> display 
    |> ignore

    sprintf "<div class='%s'></div>" markerClass
    |> HtmlString
    |> display
    |> ignore

    fun (file:string, bytesRead : int64, totalBytes: int64) ->
        let status = 
            sprintf "Downloading file [%s] -- %dKB of %.02fMB received." 
                     (file.Replace("\\","/")) bytesRead (float totalBytes/1024.0/1024.0)

        let percentage = float bytesRead / float totalBytes * 100.0

        ProgressBar.Update pbarClass status percentage
        sprintf "$('.%s') \
                    .closest('.output') \
                    .find('.output_area') \
                    .each((i,el)=>{ if (i > 1) $(el).remove(); });" markerClass
        |> Javascript              

In [ ]:
let kaggleJsonPath = "kaggle.json"
let client = new HttpClient()

{ DatasetInfo =
      { Owner = "selfishgene"
        Dataset = "historical-hourly-weather-data"
        Request = CompleteDatasetZipped }
  
  AuthorizedClient =
      kaggleJsonPath
      |> Credentials.LoadFrom
      |> Credentials.AuthorizeClient client
  
  DestinationFolder = "../Data"      
  CancellationToken = None
  ReportingCallback = Some (Report()) }
|> DownloadDatasetAsync
|> Async.AwaitTask
|> Async.RunSynchronously